# PKI Secrets Engine - Bring Your Own Root CA



In this scenario, we use Vault as a trusted Intermediate in the certificate chain for signing certificates for existing PKI infrastructure.
<img align=center src=Images/pki-img1.png width=600/>
We use Vault as Intermediate CA and keep the Organisation's Root CA offline in an airgapped env.A shorter-lived intermediate CA certificate is generated using Root CA and the intermediate CA is put this into the Vault. 

<img align=center src=Images/pki-img2.jpg/>

Clients can request Vault to issue an identity certificate, which is signed by the Intermediate CA we configured earlier.

Complete Steps span across two phases, Configuration - generally one time Intial set up and Usage, Day 1 and Day 2 activities.

<b>CONFIGURATION:</b>

1. Setting up Vault
2. Configuring the PKI Secrets Engine
    1. Enable the Engine
    2. Configure the Engine as Intermediate 
    3. Import Signed Certificate back in Vault
3. Creating a role

<b>USAGE:</b>

4. Request a certificate 

---
# Configuration

## Step#1: Setting Up Vault Client

In [2]:
export VAULT_ADDR=http://localhost:8200
export VAULT_TOKEN=root
vault login root

WARNING! The VAULT_TOKEN environment variable is set! This takes precedence
over the value set by this command. To use the value set by this command,
unset the VAULT_TOKEN environment variable or set it to the token displayed
below.

Success! You are now authenticated. The token information displayed below
is already stored in the token helper. You do NOT need to run "vault login"
again. Future Vault requests will automatically use this token.

Key                  Value
---                  -----
token                root
token_accessor       87H5mcaZNC4PPYDct163lx0c
token_duration       ∞
token_renewable      false
token_policies       ["root"]
identity_policies    []
policies             ["root"]


In [12]:
vault status

Key             Value
---             -----
Seal Type       shamir
Initialized     true
Sealed          false
Total Shares    1
Threshold       1
Version         1.9.2
Storage Type    inmem
Cluster Name    vault-cluster-3f1a29ba
Cluster ID      35c5bf70-a162-52ba-07e6-40a5aa36e777
HA Enabled      false


## Step#2: Configuring the PKI Secrets Engine

### 2a. Enable the PKI Secrets Engine

In [3]:
#Enable PKI Secrets Engine at a specified path
vault secrets enable -path="byoca-int1" pki

Success! Enabled the pki secrets engine at: byoca-int1/


In [4]:
#You may increase the TTL, if required by tuning the secrets engine. 
vault secrets tune -max-lease-ttl=8760h byoca-int1

Success! Tuned the secrets engine at: byoca-int1/


In [5]:
vault secrets list

Path                          Type         Accessor              Description
----                          ----         --------              -----------
byoca-int1/                   pki          pki_c2866551          n/a
cubbyhole/                    cubbyhole    cubbyhole_a5e84643    per-token private secret storage
identity/                     identity     identity_3f771378     identity store
openldap/                     openldap     openldap_daf63dde     n/a
pki/                          pki          pki_7bc04361          n/a
pki_int/                      pki          pki_1f2d28a7          n/a
rbbank-key/                   transit      transit_7f6fd3cf      n/a
secret/                       kv           kv_8031e0df           key/value secret storage
ssh-client-signe-non-prod/    ssh          ssh_a4138960          n/a
ssh-client-signer/            ssh          ssh_bbe10c96          n/a
ssh-user-ca/                  ssh          ssh_ae95f83c          n/a
sys/                      

### 2b: Configure the Secret Engine to be an Intermediate CA

In [6]:
vault write -format=json  byoca-int1/intermediate/generate/internal common_name="nidhimishra.cf Intermediate" | jq .data.csr -r > pki101.csr
cat pki101.csr

-----BEGIN CERTIFICATE REQUEST-----
MIICazCCAVMCAQAwJjEkMCIGA1UEAxMbbmlkaGltaXNocmEuY2YgSW50ZXJtZWRp
YXRlMIIBIjANBgkqhkiG9w0BAQEFAAOCAQ8AMIIBCgKCAQEAuSRJMcQyxECs5dzV
r7DZGXcOO/5IX77WFZiEynaKs9tXMKy0g3JQmzh4ZkEpalgt3wZCIxc5Yu7mKc3A
99qUpTIjtDI8VIkhG90wKlz/zZqkTs820t33MX2HEAmcfNP6hlcxDilkXCSZ+aLr
/Z4tuI5DFMkFJ96zYLVJp4TOANB2qPSNLduX5yLczrLMpZmSaZnDDakhcIBU7Kpx
iRpVkbWXQKY4p7R0JfU44IVdKQMZzIh9Fz8zf8VWiL5h8TLGaZrY2uk52j0rdPyL
deRMy4JCRF5wIwYhboIWCjy/kF5PMtl2fNMbdx/kra5QbBN9KyZLuHgb+BqrQI6+
hFzdcwIDAQABoAAwDQYJKoZIhvcNAQELBQADggEBAKvNJgqVZqqe1sxkL8s/BpqT
bD2UR/dpkawTDL0QSoIF0rDcQlPfZLp1zvX3zNRUIAtoNuXQdVWL/cIH2eARK0NX
gEMGwDOptDZRXm7qaV/TzM1OhyKiK2p6jn44PKcLDWQUbldFWsZhDkkUlDZG+V8u
9qP8VgpBx92C1bHjWtE5VRN2sWcNVA79qZhbc9TWEhb36/hQx1/x7PHzvukTLvU6
t7j8gCrxXfW2ywD5y44IVoxOCoxXsr1wOC6ZDqIAzaI+Rf3J50v3ArCguy8yFNdU
zCbfVe0gl6F4W1tONGT9I+TwqHxezVVMjAaUr+sFsbrjlFMXqGGRQYNtPAyCGYg=
-----END CERTIFICATE REQUEST-----


#### This generates a CSR which must be signed by organisation's root certificate.

In [7]:
openssl req -text -noout -verify -in pki101.csr

verify OK
Certificate Request:
    Data:
        Version: 1 (0x0)
        Subject: CN = nidhimishra.cf Intermediate
        Subject Public Key Info:
            Public Key Algorithm: rsaEncryption
                RSA Public-Key: (2048 bit)
                Modulus:
                    00:b9:24:49:31:c4:32:c4:40:ac:e5:dc:d5:af:b0:
                    d9:19:77:0e:3b:fe:48:5f:be:d6:15:98:84:ca:76:
                    8a:b3:db:57:30:ac:b4:83:72:50:9b:38:78:66:41:
                    29:6a:58:2d:df:06:42:23:17:39:62:ee:e6:29:cd:
                    c0:f7:da:94:a5:32:23:b4:32:3c:54:89:21:1b:dd:
                    30:2a:5c:ff:cd:9a:a4:4e:cf:36:d2:dd:f7:31:7d:
                    87:10:09:9c:7c:d3:fa:86:57:31:0e:29:64:5c:24:
                    99:f9:a2:eb:fd:9e:2d:b8:8e:43:14:c9:05:27:de:
                    b3:60:b5:49:a7:84:ce:00:d0:76:a8:f4:8d:2d:db:
                    97:e7:22:dc:ce:b2:cc:a5:99:92:69:99:c3:0d:a9:
                    21:70:80:54:ec:aa:71:89:1a:55:91:b5:97:40:a6:
         

### 2c: Import the signed certificate back in vault

In [18]:
vault write  byoca-int1/intermediate/set-signed certificate=@pki101.pem

Failed to parse K=V data: invalid key/value pair "certificate=@pki101.pem": error reading file: open pki101.pem: no such file or directory


: 1

## Step#3: Create a role

##### A role is a logical name that maps to a policy used to generate those credentials. It allows configuration parameters to control certificate common names, alternate names, the key uses that they are valid for, and more.

In [9]:
vault write byoca-int1/roles/byoca-nidhimishra-dot-cf \
        allowed_domains="nidhimishra.cf" \
        allow_subdomains=true \
        max_ttl="720h"

Success! Data written to: byoca-int1/roles/byoca-nidhimishra-dot-cf


-----

# Usage

## Step#4: Request a leaf certificate using Vault

In [19]:
vault write byoca-int1/issue/byoca-nidhimishra-dot-cf common_name="test90.nidhimishra.cf" ttl="24h" 

Error writing data to byoca-int1/issue/byoca-nidhimishra-dot-cf: Error making API request.

URL: PUT http://localhost:8200/v1/byoca-int1/issue/byoca-nidhimishra-dot-cf
Code: 400. Errors:

* unknown role: byoca-nidhimishra-dot-cf


: 2

In [95]:
vault write byoca-int1/issue/byoca-nidhimishra-dot-cf common_name="test90.nidhimishra.cf" ttl="24h" -format=json| jq .data.certificate -r > leaf-cert.crt
openssl x509 -in leaf-cert.crt -text -noout 

Error writing data to byoca-int1/issue/byoca-nidhimishra-dot-cf: Error making API request.

URL: PUT http://localhost:8200/v1/byoca-int1/issue/byoca-nidhimishra-dot-cf
Code: 500. Errors:

* 1 error occurred:
	* error fetching CA certificate: stored CA information not able to be parsed


unable to load certificate
8595631616:error:0909006C:PEM routines:get_name:no start line:crypto/pem/pem_lib.c:745:Expecting: TRUSTED CERTIFICATE


: 1

---
## Revoke certificates

##### If a certificate must be revoked, you can easily perform the revocation action which will cause the CRL to be regenerated. When the CRL is regenerated, any expired certificates are removed from the CRL.

In [96]:
vault write byoca-int1/revoke serial_number='7a:a0:59:e0:0b:17:f1:2f:a1:fa:ec:63:c6:00:22:77:0d:db:52:5e'

Error writing data to byoca-int1/revoke: Error making API request.

URL: PUT http://localhost:8200/v1/byoca-int1/revoke
Code: 500. Errors:

* 1 error occurred:
	* error fetching CA certificate: stored CA information not able to be parsed




: 2

---
## Remove Expired Certificates

##### Keep the storage backend and CRL by periodically removing certificates that have expired and are past a certain buffer period beyond their expiration time.

In [97]:
vault write byoca-int1/tidy tidy_cert_store=true tidy_revoked_certs=true

WARNING! The following warnings were returned from Vault:

  * Tidy operation successfully started. Any information from the operation
  will be printed to Vault's server logs.



---